In [166]:
from TextNormalizer import TextNormalizer
from flashtext import KeywordProcessor
import re
import pickle

In [152]:
def save_bin(fname,var):
    with open(fname,'wb') as fp:
        pickle.dump(var,fp)

In [74]:
def load_bin(fname):
    with open(fname,'rb') as fp:
        return pickle.load(fp)

In [86]:
def load_txt(fname,vec_delim = '/',phrase_delim='\n'):
    with open(fname) as f:
        return [list(filter(lambda x: x!='',elem.split(phrase_delim))) for elem in f.read().split(vec_delim)]

In [103]:
def normalize_vecs(vecs):
    return [[' '.join(TextNormalizer.normalize(phrase)) for phrase in vec] for vec in vecs]

In [104]:
def normalize_docs(docs):
    return [' '.join(TextNormalizer.normalize(doc)) for doc in docs]

In [162]:
docs = load_bin('docs.txt')

In [212]:
sdocs = [[' '.join(TextNormalizer.normalize(sent)) for sent in re.split('\. |! |\?',doc)] for doc in docs]

In [154]:
nvecs = load_bin('nvecs')
ndocs = load_bin('ndocs')

In [209]:
def locate_phrase(phrase, sentence):
    ph_words = phrase.split()
    keyword_processor = KeywordProcessor()
    for word in ph_words:
        keyword_processor.add_keyword(word)
    found_words = ' '.join(keyword_processor.extract_keywords(sentence))
    keyword_processor = KeywordProcessor()
    keyword_processor.add_keyword(phrase)
    for elem in found_words:
        found = keyword_processor.extract_keywords(found_words)
        if found:
            return found
    return []

In [219]:
def relevance(vec,doc):
    found_phrases = []
    for phrase in vec:
        for sent in doc:
            found = locate_phrase(phrase,sent)
            if found:
                found_phrases.append(found[0])
                break
    relevance = len(found_phrases)/len(vec)
    missing_phrases = set(vec).difference(set(found_phrases))
    return relevance, missing_phrases

In [220]:
relevance(nvecs[0],sdocs[0])

(0.30864197530864196,
 {'lg сплит',
  'бытовой кондиционер',
  'вентилятор',
  'домашний кондиционер',
  'инвертор кондиционер',
  'инверторный кондиционер',
  'инверторный сплит система',
  'канальный кондиционер',
  'канальный тип',
  'кассетный кондиционер',
  'колонный кондиционер',
  'комнатный кондиционер',
  'кондиционер 12',
  'кондиционер daikin',
  'кондиционер fujitsu',
  'кондиционер gree',
  'кондиционер hitachi',
  'кондиционер lg',
  'кондиционер mitsubishi',
  'кондиционер panasonic',
  'кондиционер samsung',
  'кондиционер акция',
  'кондиционер воздуховод',
  'кондиционер воронеж',
  'кондиционер дайкина',
  'кондиционер дом',
  'кондиционер дёшево',
  'кондиционер киев купить',
  'кондиционер киев цена',
  'кондиционер лджи',
  'кондиционер лессара',
  'кондиционер лж',
  'кондиционер митсубись',
  'кондиционер недорого москва',
  'кондиционер оптом',
  'кондиционер отзыв',
  'кондиционер официальный сайт',
  'кондиционер панасоник',
  'кондиционер самсунг',
  'конди

### Tf-idf

In [17]:
nvecs = [TextNormalizer.normalize(text) for text in vecs]

In [18]:
ndocs = [TextNormalizer.normalize(doc) for doc in docs]

In [19]:
dct = Dictionary(nvecs)

In [20]:
vec_corpus = [dct.doc2bow(vec) for vec in nvecs]

In [21]:
doc_corpus = [dct.doc2bow(doc) for doc in ndocs]

In [23]:
model = TfidfModel(vec_corpus,dictionary=dct)

In [24]:
tf_vecs = [model[vec] for vec in vec_corpus]

In [25]:
tf_docs = [model[doc] for doc in doc_corpus]

In [26]:
index = similarities.SparseMatrixSimilarity(tf_vecs, num_features=dct.num_pos)

In [36]:
index[tf_docs[3]]

array([0.07533741, 0.06170711, 0.10711007, 0.78122646], dtype=float32)